In [ ]:
import json

# Load the JSON files
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# File paths
categories_path = '/Users/marian.dumitrascu/Library/CloudStorage/Dropbox/work-P/4urspace/4urspace-next-lll/temp_data/categories.json'
cities_path = '/Users/marian.dumitrascu/Library/CloudStorage/Dropbox/work-P/4urspace/4urspace-next-lll/temp_data/cities.json'
malls_path = '/Users/marian.dumitrascu/Library/CloudStorage/Dropbox/work-P/4urspace/4urspace-next-lll/temp_data/malls.json'
vendors_path = '/Users/marian.dumitrascu/Library/CloudStorage/Dropbox/work-P/4urspace/4urspace-next-lll/temp_data/vendors.json'

# Load data
categories_data = load_json(categories_path)
cities_data = load_json(cities_path)
malls_data = load_json(malls_path)
vendors_data = load_json(vendors_path)

# Display sample data from each file to understand structure
# sample_data = {
#     "categories": categories_data[:5],  # Show first 5 records
#     "cities": cities_data[:5],
#     "malls": malls_data[:5],
#     "vendors": vendors_data[:5]
# }
# sample_data

sample_data2 = {
    "categories": categories_data,
    "cities": cities_data,
    "malls": malls_data,
    "vendors": vendors_data
}
sample_data2


In [2]:
import pandas as pd

# # Assuming categories_data is a list of dictionaries
# categories_data = [{'cid': '32', 'pid': '230', 'lid': '20263', 'cgid': '3', 'cgname': "Women's Apparel"}, {'cid': '32', 'pid': '230', 'lid': '20263', 'cgid': '4', 'cgname': "Women's Accessory"}, {'cid': '32', 'pid': '747', 'lid': '20089', 'cgid': '3', 'cgname': "Women's Apparel"}, {'cid': '32', 'pid': '747', 'lid': '20089', 'cgid': '2', 'cgname': "Men's Apparel"}, {'cid': '32', 'pid': '4167', 'lid': '20174', 'cgid': '18', 'cgname': 'Jewelry & Watches'}]

# Convert to columnar data
categories_df = pd.json_normalize(categories_data)
cities_df = pd.json_normalize(cities_data)
malls_df = pd.json_normalize(malls_data)
vendors_df = pd.json_normalize(vendors_data)

categories_df.to_csv('temp_data/categories.csv', index=False)
cities_df.to_csv('temp_data/cities.csv', index=False)
malls_df.to_csv('temp_data/malls.csv', index=False)
vendors_df.to_csv('temp_data/vendors.csv', index=False)

# # Print the resulting dataframe
# print(categoried_df)


In [ ]:
# To analyze Category Distribution Across Cities, we need to link categories to cities via malls or directly through location IDs if applicable.

# Convert lists to dictionaries for faster access
cities_dict = {city['cid']: city for city in cities_data}
malls_dict = {mall['lid']: mall for mall in malls_data}

# Prepare a data structure to hold category distribution by city
category_distribution = {}

# Iterate over categories to populate the distribution
for category in categories_data:
    city_id = malls_dict.get(category['lid'], {}).get('city', 'Unknown')  # Get city from mall location ID
    city_name = cities_dict.get(city_id, {}).get('cn', 'Unknown')  # Get city name from city ID

    if city_name not in category_distribution:
        category_distribution[city_name] = {}

    category_name = category['cgname']
    if category_name in category_distribution[city_name]:
        category_distribution[city_name][category_name] += 1
    else:
        category_distribution[city_name][category_name] = 1

# Output the top cities and their category distributions
sorted_category_distribution = {k: v for k, v in sorted(category_distribution.items(), key=lambda item: len(item[1]), reverse=True)}
sorted_category_distribution
